# Interdiction de plus courts chemins

MPRO 2018/2018 - RORT

*Guillaume CROGNIER, Guillaume DALLE, Olivier RIGAL*

## 2. Deuxième PL

Installer les packages nécessaires

In [17]:
using Pkg
# Pkg.update()
# Pkg.add("JuMP")
# Pkg.add("Cbc")
# Pkg.add("LightGraphs")

Importer les packages

In [18]:
using JuMP
using Cbc
using LightGraphs, SimpleWeightedGraphs
using Random

Importer le générateur d'instances

In [19]:
include("instances_io.jl");

### 2.1. Deuxième programme

Génération de l'instance

In [20]:
Random.seed!(63);
l = 5 # number of lines in the grid-like graph
c = 5 # number of columns in the grid-like graph
k = 10 # number of edges the adversary can penalize
maxc = 5 # range of initial edge cost
maxd = 50 # range of penalized edge cost
sv = generate(l, c, k, maxc, maxd);
arcs = []
for u in 1:sv.n, v in 1:sv.n
    if sv.adj[u][v]
        push!(arcs, (u,v))
    end
end

Création du problème maître, ajout de l'objectif et des variables

In [21]:
master = Model(solver=CbcSolver())

x = @variable(master, x[arcs], Bin)
z = @variable(master, z<=10e7)

s, t = 1, sv.n # source and target nodes
@objective(master, Max, z)

print(master)

Max z
Subject to
 x[i] in {0,1} for all i in {(1, 2),(1, 3),..,(25, 27),(26, 27)}
 z <= 1.0e8


Ajout des contraintes

Pas plus de k arcs sélectionnés

In [22]:
@constraint(master, max_k_arcs, sum(x[(u,v)] for (u,v) in arcs) <= k)

x[(1, 2)] + x[(1, 3)] + x[(1, 4)] + x[(1, 5)] + x[(1, 6)] + x[(2, 7)] + x[(2, 8)] + x[(3, 7)] + x[(3, 8)] + x[(3, 9)] + x[(4, 8)] + x[(4, 9)] + x[(4, 10)] + x[(5, 9)] + x[(5, 10)] + x[(5, 11)] + x[(6, 10)] + x[(6, 11)] + x[(7, 8)] + x[(7, 12)] + x[(7, 13)] + x[(8, 7)] + x[(8, 9)] + x[(8, 12)] + x[(8, 13)] + x[(8, 14)] + x[(9, 8)] + x[(9, 10)] + x[(9, 13)] + x[(9, 14)] + x[(9, 15)] + x[(10, 9)] + x[(10, 11)] + x[(10, 14)] + x[(10, 15)] + x[(10, 16)] + x[(11, 10)] + x[(11, 15)] + x[(11, 16)] + x[(12, 13)] + x[(12, 17)] + x[(12, 18)] + x[(13, 12)] + x[(13, 14)] + x[(13, 17)] + x[(13, 18)] + x[(13, 19)] + x[(14, 13)] + x[(14, 15)] + x[(14, 18)] + x[(14, 19)] + x[(14, 20)] + x[(15, 14)] + x[(15, 16)] + x[(15, 19)] + x[(15, 20)] + x[(15, 21)] + x[(16, 15)] + x[(16, 20)] + x[(16, 21)] + x[(17, 18)] + x[(17, 22)] + x[(17, 23)] + x[(18, 17)] + x[(18, 19)] + x[(18, 22)] + x[(18, 23)] + x[(18, 24)] + x[(19, 18)] + x[(19, 20)] + x[(19, 23)] + x[(19, 24)] + x[(19, 25)] + x[(20, 19)] + x[(20, 21)] +

Sous-problème des plus courts chemins

In [23]:
function PPC(sv::Data, x::JuMP.JuMPArray{Float64,1,Tuple{Array{Any,1}}})
    g = SimpleWeightedDiGraph(sv.n)
    for (u, v) in arcs
        penalized = x[(u,v)]
        cost = sv.c[u][v] + penalized * sv.d[u][v]
        add_edge!(g, u, v, cost)
    end
    sp = dijkstra_shortest_paths(g, 1)
    path_cost = sp.dists[sv.n]
    path = []
    next_node = sv.n
    while next_node != 0
        prepend!(path, next_node)
        next_node = sp.parents[next_node]
    end
    path2=[]
    for i in 1:length(path)-1
        (u,v)=(path[i],path[i+1])
        push!(path2,(u,v))
    end
    return path_cost, path2
end

PPC (generic function with 1 method)

Boucle principale

In [24]:
obj=10e9
spb=0

while obj > spb+0.5

    solve(master, relaxation=false)

    x_opt = getvalue(x)
            
    obj = getobjectivevalue(master)

    spb, y = PPC(sv,x_opt) #y est une liste d'arcs correspondant au plus court chemin dans le graphe induit par x*
    
    print(y)
    
    println(spb)
        
    @constraint(master, sum(sv.c[u][v] + sv.d[u][v]*x[(u,v)] for (u,v) in y) >= z)
    
    println(obj)
    
end

print(obj)

Any[(1, 5), (5, 9), (9, 15), (15, 20), (20, 25), (25, 27)]8.0
1.0e8
Any[(1, 6), (6, 11), (11, 10), (10, 14), (14, 18), (18, 22), (22, 27)]9.0
198.0
Any[(1, 2), (2, 8), (8, 14), (14, 19), (19, 25), (25, 27)]11.0
157.0
Any[(1, 2), (2, 8), (8, 13), (13, 17), (17, 22), (22, 27)]11.0
128.0
Any[(1, 6), (6, 11), (11, 15), (15, 20), (20, 24), (24, 27)]12.0
100.0
Any[(1, 5), (5, 10), (10, 14), (14, 18), (18, 19), (19, 25), (25, 27)]12.0
89.0
Any[(1, 2), (2, 8), (8, 12), (12, 17), (17, 22), (22, 27)]13.0
89.0
Any[(1, 2), (2, 7), (7, 13), (13, 18), (18, 22), (22, 27)]12.0
76.0
Any[(1, 3), (3, 9), (9, 15), (15, 21), (21, 25), (25, 27)]12.0
72.0
Any[(1, 4), (4, 10), (10, 14), (14, 19), (19, 24), (24, 27)]15.0
63.0
Any[(1, 2), (2, 7), (7, 13), (13, 18), (18, 24), (24, 27)]16.0
59.0
Any[(1, 2), (2, 7), (7, 13), (13, 18), (18, 23), (23, 27)]17.0
59.0
Any[(1, 6), (6, 11), (11, 15), (15, 21), (21, 25), (25, 27)]11.0
52.0
Any[(1, 5), (5, 9), (9, 14), (14, 18), (18, 22), (22, 27)]10.0
51.0
Any[(1, 5), (5,